<a href="https://colab.research.google.com/github/sashkoangelov/NLP_final_project/blob/main/NLP_final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
pip install datasets

In [39]:
pip install transformers[torch]

In [40]:
pip install accelerate -U


In [41]:
import pandas as pd
import torch
from transformers import RobertaTokenizer, RobertaTokenizerFast, RobertaForQuestionAnswering
from torch.utils.data import Dataset, DataLoader
from datasets import Dataset

In [42]:
train = pd.read_parquet("/content/drive/MyDrive/NLP/train-00000-of-00001.parquet")
validation = pd.read_parquet("/content/drive/MyDrive/NLP/validation-00000-of-00001.parquet")

In [43]:
train.head()

,id,title,context,question,answers
0,56be85543aeaaa14008c9063,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce start becoming popular?,"{'text': ['in the late 1990s'], 'answer_start'..."
1,56be85543aeaaa14008c9065,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What areas did Beyonce compete in when she was...,"{'text': ['singing and dancing'], 'answer_star..."
2,56be85543aeaaa14008c9066,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce leave Destiny's Child and bec...,"{'text': ['2003'], 'answer_start': [526]}"
3,56bf6b0f3aeaaa14008c9601,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In what city and state did Beyonce grow up?,"{'text': ['Houston, Texas'], 'answer_start': [..."
4,56bf6b0f3aeaaa14008c9602,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In which decade did Beyonce become famous?,"{'text': ['late 1990s'], 'answer_start': [276]}"


In [44]:
def preprocess_data(row):
    # Check if 'text' and 'answer_start' are not empty
    if len(row["answers"]["text"]) > 0 and len(row["answers"]["answer_start"]) > 0:
        return {
            "context": row["context"],
            "question": row["question"],
            "answers": {
                "text": row["answers"]["text"][0],  # Use the first answer
                "answer_start": row["answers"]["answer_start"][0]
            }
        }
    else:
        # Handle rows with empty answers - you might choose to return None or a default value
        return None

In [45]:
train_preprocessed = train.apply(preprocess_data, axis=1)
validation_preprocessed = validation.apply(preprocess_data, axis=1)

### Converting to Hugging face dataset

In [46]:
train_dataset = Dataset.from_pandas(train)
validation_dataset = Dataset.from_pandas(validation)

In [47]:
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')

def tokenize_function(examples):
    return tokenizer(examples["question"], examples["context"], truncation=True, padding="max_length", max_length=512)

tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_validation = validation_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/130319 [00:00<?, ? examples/s]

Map:   0%|          | 0/11873 [00:00<?, ? examples/s]

In [48]:
from transformers import RobertaForQuestionAnswering, TrainingArguments, Trainer

model = RobertaForQuestionAnswering.from_pretrained('roberta-base')

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=12,
    per_device_eval_batch_size=12,
    num_train_epochs=2,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_validation,
    tokenizer=tokenizer,
)

trainer.train()

Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at roberta-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`